In [0]:
!pip install -U -q PyDrive

In [0]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!mkdir data

In [0]:
file_id = '1kFMUiXtrlw5B6WTjJ-mUm4-STV28Zsvn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/auto.csv')

In [0]:
df = pd.read_csv('data/auto.csv', index_col='ID')

In [7]:
df.head()

,CarNumber,Make_n_model,Refund,Fines,History
ID,,,,,
0,Y163O8161RUS,Ford Focus,2.0,3200.0,NaN
1,E432XX77RUS,Toyota Camry,1.0,6500.0,NaN
2,7184TT36RUS,Ford Focus,1.0,2100.0,NaN
3,X582HE161RUS,Ford Focus,2.0,2000.0,NaN
4,E34877152RUS,Ford Focus,2.0,6100.0,NaN


In [8]:
df.count()

CarNumber       931
Make_n_model    931
Refund          914
Fines           869
History          82
dtype: int64

In [0]:
df = df.drop_duplicates(keep='last')

In [10]:
df.count()

CarNumber       825
Make_n_model    825
Refund          808
Fines           764
History          82
dtype: int64

In [11]:
df.isnull().sum()

CarNumber         0
Make_n_model      0
Refund           17
Fines            61
History         743
dtype: int64

In [0]:
df = df.dropna(axis=1, thresh=500)

In [13]:
df.isnull().sum()

CarNumber        0
Make_n_model     0
Refund          17
Fines           61
dtype: int64

In [0]:
df['Refund'] = df['Refund'].fillna(method='pad')

In [15]:
df.isnull().sum()

CarNumber        0
Make_n_model     0
Refund           0
Fines           61
dtype: int64

In [0]:
mean_fine = df['Fines'].mean()
df['Fines'] = df['Fines'].fillna(mean_fine)

In [17]:
df.isnull().sum()

CarNumber       0
Make_n_model    0
Refund          0
Fines           0
dtype: int64

In [0]:
df['Make'] = df.apply(lambda row: row['Make_n_model'].split()[0], axis=1)
df['Model'] = df.apply(lambda row: row['Make_n_model'].split()[1]
                       if len(row['Make_n_model'].split()) > 1 else np.nan,
                       axis=1)
df = df.drop(labels=['Make_n_model'], axis=1)

In [19]:
df.head()

,CarNumber,Refund,Fines,Make,Model
ID,,,,,
0,Y163O8161RUS,2.0,3200.0,Ford,Focus
1,E432XX77RUS,1.0,6500.0,Toyota,Camry
2,7184TT36RUS,1.0,2100.0,Ford,Focus
3,X582HE161RUS,2.0,2000.0,Ford,Focus
5,92918M178RUS,1.0,5700.0,Ford,Focus


In [20]:
df.count()

CarNumber    825
Refund       825
Fines        825
Make         825
Model        813
dtype: int64

In [0]:
df.to_json('data/auto.json', orient='records')

In [0]:
uploaded = drive.CreateFile({'title': 'auto.json'})
uploaded.SetContentFile('data/auto.json')
uploaded.Upload()